# Materials

In PyMoosh, the part dealing with materials can be considered almost independant, and can easily be re-used in other programs -- especially the database. 

Here we show a little bit how this works and what you can do with it. 

In order to make things simple for the user, when a Structure is defined the informations which are given are used to generate an object which belongs to the `Material` class.

In [1]:
from PyMoosh import *

air = Material(1.)

`air`is thus a material, which means you can access its permittivity and permeability easily...

In [2]:
# Wavelength in nanometer
wavelength = 600
epsilon = air.get_permittivity(wavelength)
mu = air.get_permeability(wavelength)
print("epsilon = ",epsilon,"\nmu =",mu)

epsilon =  1.0 
mu = 1.0


This was easy. Now we can declare another material but may want to specify its permeability, because it is different from one. It is still a non-dispersive material which we could call `metamaterial` for instance. It has a magnetic response, but it is not dispersive, hence the name of the class `MagneticND`.

In [3]:
metamaterial = MagneticND(-1.,-2.)
epsilon = metamaterial.get_permittivity(wavelength)
mu = metamaterial.get_permeability(wavelength)
print("epsilon = ",epsilon,"\nmu =",mu)

epsilon =  -1.0 
mu = -2.0


Now, you may want to be able to define a material, like an ideal metal which has a permittivity described by a simple Drude model. Well, the Drude model is not implemented yet, but it's a good idea. You can still always do this, if the plasma wavelength is 125 nm for instance (typically the case for gold and silver) : 

In [4]:
def drude(wavelength):
    return 1 - wavelength**2 / 125 **2

And then define a material using the class `CustomFunction` which is exactly meant for such a case (but where the permeability is assumed to be unity). You can check that in this case the material is dispersive. 

In [5]:
metal = CustomFunction(drude)
wavelength = 400
epsilon = metal.get_permittivity(wavelength)
mu = metal.get_permeability(wavelength)
print("epsilon = ",epsilon,"\nmu =",mu)

epsilon =  -9.24 
mu = 1.0


And look, it is still a material, because if you ask : 

In [6]:
issubclass(metal.__class__,Material)

True

Any of those materials can be provided to declare a class `Structure`, it will be used as is. 

If you have actual data, a list of wavelengths and a list of corresponding permittivities, you can define a material which corresponds to that too, thanks to the `ExpData` class (yes, we assume it is because you have experimental data). 

In [7]:
import numpy as np 
wavelength_list = np.linspace(300,800,100)
# Let us use our Drude model declared above to generate the corresponding permittivities :
permittivities = drude(wavelength_list)
metal2 = ExpData(wavelength_list,permittivities)

wavelength  = 500
epsilon = metal2.get_permittivity(wavelength)
mu = metal2.get_permeability(wavelength)
print("The interpolation provides : \nepsilon = ",epsilon,"\nmu =",mu)
print("Drude model provides : epsilon =",metal.get_permittivity(500))

The interpolation provides : 
epsilon =  (-15.000391796755434+0j) 
mu = 1.0
Drude model provides : epsilon = -15.0


As you can see, the values are not exactly the same. First for experimental data, it is always assumed epsilon is complex. Then, it is an interpolation between experimental points, so this is not as accurate as an analytic formula.

In [8]:
list_of_materials = [air,metamaterial,metal,metal2]
multilayer = Structure(list_of_materials,[0,1],[100,100])


List of materials:
Object: Material
Object: MagneticND
Object: CustomFunction
Object: ExpData


If you need, you can modify the list of materials, but since Structure makes sure it is a list of objects, you have to insert or modify the material objects directly. 


In [9]:
new_material = Material(2.)
multilayer.materials.append(new_material)
print(multilayer.materials)



[<PyMoosh.materials.Material object at 0x7f3140607010>, <PyMoosh.materials.MagneticND object at 0x7f3106222f20>, <PyMoosh.materials.CustomFunction object at 0x7f3106223490>, <PyMoosh.materials.ExpData object at 0x7f3106222590>, <PyMoosh.materials.Material object at 0x7f31062235e0>]


In [10]:
multilayer.materials[0].permittivity = 1.3
